In [ ]:
import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from astroquery.gaia import Gaia

Get a random sampling of 100000 stars within 1 kpc

In [ ]:
query = f"""
SELECT TOP 100000 
    gaia.source_id, gaia.phot_g_mean_mag, gaia.bp_rp, gaia.parallax 
FROM 
    gaiadr2.gaia_source as gaia
INNER JOIN
    gaiadr2.ruwe as ruwe ON gaia.source_id = ruwe.source_id
WHERE 
    gaia.parallax > 1 AND
    gaia.parallax_over_error > 5 AND
    gaia.phot_g_mean_mag < 20.7 AND
    gaia.bp_rp BETWEEN -1 AND 5 AND
    ruwe.ruwe < 1.4
order by gaia.random_index
"""

job = Gaia.launch_job_async(query)
random_tbl = at.QTable(job.get_results())

In [ ]:
cmd_bins = (np.arange(-1, 5+1e-3, 0.04),
            np.arange(0, 20.7+1e-3, 0.08))

fig, ax = plt.subplots(figsize=(6, 6))
ax.hist2d(random_tbl['bp_rp'].value,
          random_tbl['phot_g_mean_mag'].value,
          bins=cmd_bins, 
          norm=mpl.colors.LogNorm())

ax.set_xlim(-1, 5)
ax.set_ylim(20.7, 0)

ax.set(xlabel=r'$G_{\rm BP}-G_{\rm RP}$ [mag]',
       ylabel=r'$G$ [mag]')

fig.tight_layout()

In [ ]:
distmod = coord.Distance(parallax=random_tbl['parallax']).distmod
M_G = random_tbl['phot_g_mean_mag'] - distmod

In [ ]:
abs_cmd_bins = (np.arange(-1, 5+1e-3, 0.04),
                np.arange(-3, 17+1e-3, 0.08))

fig, ax = plt.subplots(figsize=(6, 6))
ax.hist2d(random_tbl['bp_rp'].value,
          M_G.value,
          bins=abs_cmd_bins, 
          norm=mpl.colors.LogNorm())

ax.set_xlim(-1, 5)
ax.set_ylim(17, -3)

ax.set(xlabel=r'$G_{\rm BP}-G_{\rm RP}$ [mag]',
       ylabel=r'$M_G$ [mag]')

fig.tight_layout()

In [ ]:
abs_cmd_bins = (np.arange(-1, 5+1e-3, 0.04),
                np.arange(-3, 17+1e-3, 0.08))

fig, ax = plt.subplots(figsize=(6, 6))
ax.hist2d(random_tbl['bp_rp'].value,
          M_G.value,
          bins=abs_cmd_bins, 
          norm=mpl.colors.LogNorm())

ax.set_xlim(-1, 5)
ax.set_ylim(17, -3)

ax.set(xlabel=r'$G_{\rm BP}-G_{\rm RP}$ [mag]',
       ylabel=r'$M_G$ [mag]')

fig.tight_layout()

In [ ]:
parallax_limit = (100*u.pc).to(u.milliarcsecond, u.parallax())

query = f"""
SELECT source_id, phot_g_mean_mag, bp_rp, parallax 
FROM gaiadr2.gaia_source
WHERE parallax > {parallax_limit.value}
"""

This will take ~1 minute to execute and download:

In [ ]:
job = Gaia.launch_job_async(query)

In [ ]:
tbl = job.get_results()
len(tbl)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 6))
ax.plot(tbl['bp_rp'],
        tbl['phot_g_mean_mag'],
        marker='o', mew=0, ms=1.5, ls='none')

ax.set_xlim(-1, 5)
ax.set_ylim(20.7, 0)